<a href="https://colab.research.google.com/github/hrnrhty/my-vae-nnabla/blob/main/step2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 2 - 入力画像と VAE 生成画像の比較

Step 2 では、Step 1 で学習した結果を用いて VAE (Variational Auto Encoder) の推論を実行します。VAE で推論を実行すると画像が生成されます。生成された画像が入力画像と比較してどのように違うのか、実際に画像を表示して確認していきます。

なお、このステップを実行するには Step 1 の学習結果が必須です。まだ Step 1 を実行していない場合は、先に Step 1 を最後まで実行して学習結果を準備してください。Step 1 のノートブックは以下のバナーから開くことができます。  
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hrnrhty/my-vae-nnabla/blob/main/step1.ipynb)

また、推論は学習に比べて処理に時間がかからないため、このステップでは GPU アクセラレーションを使用せず、CPU のみで処理を実行していきます。

## Google Drive のマウント

Step 1 の学習結果を読み込むため Google Drive をマウントします。

> Note: 先に Step 1 を最後まで実行しておいてください。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## NNabla のインストール

本ステップでは GPU アクセラレーションを使用しないので、通常の [NNabla](https://github.com/sony/nnabla) だけをインストールします（CUDA 版はインストールしません）。

ここでは、2022 年 2 月 2 日時点、最新バージョンである v1.25.0 をインストールしますが、おもむろにインストールを実行すると `pip` コマンドの実行結果の中にエラーメッセージが表示されます。依存パッケージの中に、Google Colab のランタイムにはじめからインストールされているものがあり、そのバージョンが NNabla が要求するバージョンよりも新しいバージョンであることが原因です。

エラーが表示されても NNabla のインストールに成功していれば以降のコードは問題なく実行できることがほとんどですが、ここでは NNabla が要求するバージョンのパッケージを手動インストールすることによりエラーを回避します。

下記、1 行目のコマンドは NNabla の依存パッケージの一部をバージョン指定でインストールします。2 行目のコマンドは NNabla をインストールします。

どのようなエラーメッセージが表示されるのか気になる方は、1 行目をコメントアウトして実行してみてください。一度実行済の場合は、Google Colab のランタイムを出荷設定にリセットしてから実行してください。

In [ ]:
!pip install urllib3==1.25.11 folium==0.2.1
!pip install nnabla==1.25.0

## nnabla-examples のクローン

Step 1 同様、[nnabla-examples](https://github.com/sony/nnabla-examples) v1.25.0 をクローンします。

In [ ]:
!if [ -d nnabla-examples ]; then rm -rf nnabla-examples; fi
!git clone https://github.com/sony/nnabla-examples.git -b v1.25.0 --depth 1

## VAE 関数の読み込み

[nnabla-examples](https://github.com/sony/nnabla-examples) の `vae.py` およびその他 Helper 関数を利用するため、カレントディレクトリを移動します。

In [ ]:
%cd 'nnabla-examples/image-classification/mnist-collection'

`vae.py` 内で提供されている関数 `vae` は `loss` しか返しません。そこで、推論結果も返すように `vae.py` を書き換えます。

In [ ]:
!sed -i -e 's/return loss/return loss, prob/g' vae.py

推論結果 `prob` も返すように改造した関数 `vae` をインポートします。

In [ ]:
from vae import vae

## 学習結果の読み込み

後ほど使用する module も含め、ここでまとめてインポートします。

In [ ]:
import nnabla as nn
import nnabla.functions as F
from mnist_data import data_iterator_mnist

Step 1 で保存した学習済みパラメータを読み込みます。

In [ ]:
_ = nn.load_parameters('/content/drive/MyDrive/my-vae-nnabla/step1/params_060000.h5')

## 推論の実行

学習済モデルが準備できたので、推論を実行してみます。下記のコードでは、変数 `x` を作成し、[MNIST](http://yann.lecun.com/exdb/mnist/) データセットからランダムに10個の画像を取り出し、`x` に格納しています。そして、関数 `vae` の推論結果に `sigmoid` 関数を適用してデータの値域を (0, 1.0) の区間に正規化し、最終的な推論結果とするよう定義しています。最後の行で `forward()` をコールし、推論を実行しています。

In [ ]:
shape_x = (1, 28, 28)
shape_z = (50,)
x = nn.Variable((10,) + shape_x)

loss, prob = vae(x, shape_z, test=True)
di_t = data_iterator_mnist(10, False)
x.d, _ = di_t.next()
prob = F.sigmoid(prob)
prob.forward()

GPU アクセラレーションは使用していませんが、あっという間に推論が完了しましたね。

それでは、入力画像とそれに対応する推論結果（生成された画像）を表示してみましょう。ここでは、画像の表示に `matplotlib` を使用します。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

8 枚目（index = 7）の入力画像を表示してみます。

ここで、入力画像の値域は [0, 255] になっています。このままでも画像の表示はできますが、あとで生成画像と比較するため 256 で割って [0, 1) の区間に正規化します。

In [ ]:
img_num = 7

In [ ]:
input_img = x.d[img_num][0] / 256.
print('---- 入力画像 ----')
print('Min. value =', input_img.min())
print('Max. value =', input_img.max())
plt.imshow(input_img, cmap='gray')
plt.show()

続いて、推論の結果、VAE が生成した画像を表示してみます。

In [ ]:
output_img = prob.d[img_num][0]
print('---- VAE 生成画像 ----')
print('Min. value =', output_img.min())
print('Max. value =', output_img.max())
plt.imshow(output_img, cmap='gray')
plt.show()

ちゃんと "2" と判別ができる画像が生成されています！しかしよく見ると、入力画像とは少し異なる点もあるようです。

様々な手書き数字を学習した VAE は、平均的な "2" の生成を試みるようにトレーニングされています。そのため、入力画像に見られた上部の黒い点が消えたり、下部の線の湾曲が直線に近づいたり、全体的に滑らかになった印象があります。

それでは最後に、入力画像と生成画像の差の絶対値をヒートマップ化して、入力画像の上に重ねて表示してみましょう。

In [ ]:
diff = output_img - input_img
abs_diff = abs(diff)
score = sum(sum(abs_diff))
print('---- 入力画像に VAE 生成画像との差分を重畳 ----')
print('Sum of absolute values of difference =', score)
print('Min. diff. (abs) =', abs_diff.min())
print('Max. diff. (abs) =', abs_diff.max())
plt.imshow(input_img, cmap='gray')
diff_img = plt.imshow(abs_diff, cmap='jet', alpha=0.5)
plt.colorbar(diff_img)
plt.show()

他の数字も同様に確認してみましょう。

In [ ]:
img_num = 8

In [ ]:
input_img = x.d[img_num][0] / 256.
print('---- 入力画像 ----')
print('Min. value =', input_img.min())
print('Max. value =', input_img.max())
plt.imshow(input_img, cmap='gray')
plt.show()

In [ ]:
output_img = prob.d[img_num][0]
print('---- VAE 生成画像 ----')
print('Min. value =', output_img.min())
print('Max. value =', output_img.max())
plt.imshow(output_img, cmap='gray')
plt.show()

In [ ]:
diff = output_img - input_img
abs_diff = abs(diff)
score = sum(sum(abs_diff))
print('---- 入力画像に VAE 生成画像との差分を重畳 ----')
print('Sum of absolute values of difference =', score)
print('Min. diff. (abs) =', abs_diff.min())
print('Max. diff. (abs) =', abs_diff.max())
plt.imshow(input_img, cmap='gray')
diff_img = plt.imshow(abs_diff, cmap='jet', alpha=0.5)
plt.colorbar(diff_img)
plt.show()